In [163]:
import itk
print(itk.__version__)

5.4.0


In [164]:
# Paths
Data = 'Data/'

scan1_name = 'case6_gre1'
scan2_name = 'case6_gre2'

In [165]:
def getFrom(path):
    scan = itk.imread(path)
    caster = itk.CastImageFilter[scan, itk.Image[itk.F, 3]].New()
    caster.SetInput(scan)
    caster.Update()
    scan = caster.GetOutput()
    
    img = itk.GetArrayFromImage(scan)
    min_val = img.min()
    max_val = img.max()
    
    img = 255 * (img - min_val) / (max_val - min_val)
    
    return itk.GetImageFromArray(img), img

scan1, img1 = getFrom(Data + scan1_name + ".nrrd")
scan2, img2 = getFrom(Data + scan2_name + ".nrrd")

print(img1.shape, img2.shape)

(176, 256, 256) (176, 256, 256)


In [166]:
# Scan 1 points
true_point_1 = [49, 82, 69]
false_point_1 = [49, 82, 90]

print("scan1 - true :", scan1[true_point_1[0], true_point_1[1], true_point_1[2]])
print("scan1 - false :", scan1[false_point_1[0], false_point_1[1], false_point_1[2]])

print()

# Scan 2 points
true_point_2 = [49, 82, 82]
false_point_2 = [49, 82, 103]

print("scan2 - true :", scan2[true_point_2[0], true_point_2[1], true_point_2[2]])
print("scan2 - false :", scan2[false_point_2[0], false_point_2[1], false_point_2[2]])

scan1 - true : 34.85294
scan1 - false : 48.97059

scan2 - true : 136.77948
scan2 - false : 53.26419


In [167]:
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

def plot_slices(slice_index):
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.title("Scan 1 - Sagittal")
    plt.scatter(true_point_1[1], true_point_1[2], color='yellow', s=1)
    plt.scatter(false_point_1[1], false_point_1[2], color='red', s=1)
    plt.imshow(img1[slice_index, :, :], cmap='gray')
    
    plt.subplot(1, 2, 2)
    plt.title("Scan 2 - Sagittal")
    plt.scatter(true_point_2[1], true_point_2[2], color='yellow', s=1)
    plt.scatter(false_point_2[1], false_point_2[2], color='red', s=1)
    plt.imshow(img2[slice_index, :, :], cmap='gray')
    
    plt.show()

# Define the slider
slice_slider = IntSlider(min=0, max=img1.shape[0]-1, step=1, value=49)

# Use the interact function to update the plot with the slider
interact(plot_slices, slice_index=slice_slider)

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


<function __main__.plot_slices(slice_index)>

In [171]:
def segmentation(scan, seedX=82, seedY=69, seedZ=49, lower=20, upper=200):
    #smoother = itk.GradientAnisotropicDiffusionImageFilter.New(Input=scan, NumberOfIterations=20, TimeStep=0.04, ConductanceParameter=3)
    #smoother.Update()
    
    connected_threshold = itk.ConnectedThresholdImageFilter.New(scan)#smoother.GetOutput())
    
    connected_threshold.SetReplaceValue(255)
    connected_threshold.SetLower(lower)
    connected_threshold.SetUpper(upper)
    connected_threshold.SetSeed((seedX, seedY, seedZ))
    connected_threshold.Update()
    segmented_image = connected_threshold.GetOutput()

    return segmented_image

In [172]:
segmented_image1 = segmentation(scan1)
segmented_image2 = segmentation(scan2)

In [173]:
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

def plot_slices(slice_index):
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.title("Scan 1 - Sagittal")
    plt.imshow(segmented_image1[slice_index, :, :], cmap='gray')
    
    plt.subplot(1, 2, 2)
    plt.title("Scan 2 - Sagittal")
    plt.imshow(segmented_image2[slice_index, :, :], cmap='gray')
    
    plt.show()

# Define the slider
slice_slider = IntSlider(min=0, max=segmented_image1.shape[0]-1, step=1, value=49)

# Use the interact function to update the plot with the slider
interact(plot_slices, slice_index=slice_slider)

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


<function __main__.plot_slices(slice_index)>